# Project Kayak

L'équipe marketing a besoin d'aide pour un nouveau projet. Après avoir effectué des recherches auprès des utilisateurs, l'équipe a découvert que 70 % de leurs utilisateurs qui planifient un voyage aimeraient avoir plus d'informations sur la destination vers laquelle ils se rendent .

De plus, les recherches sur les utilisateurs montrent que les gens ont tendance à se méfier des informations qu'ils lisent s'ils ne connaissent pas la marque qui a produit le contenu.

Par conséquent, Kayak Marketing Team aimerait créer une application qui recommandera où les gens devraient planifier leurs prochaines vacances. L'application doit être basée sur des données réelles concernant :

>* Temps

>* Hôtels dans la région

L'application devrait alors être en mesure de recommander les meilleures destinations et hôtels en fonction des variables ci-dessus à tout moment.

**Objectifs**

Comme le projet vient de démarrer, votre équipe ne dispose d'aucune donnée pouvant être utilisée pour créer cette application. Ainsi, votre travail consistera à :

>* Extraire les données des destinations
>* Obtenez les données météorologiques de chaque destination
>* Obtenir des informations sur les hôtels de chaque destination
>* Stocker toutes les informations dans un Data lake
>* Extraire, transformer et charger les données nettoyées 

# import librairies

In [1]:
import requests
import json 
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import os 
import logging
from datetime import datetime
from bs4 import BeautifulSoup
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

pio.renderers.default = "iframe"
!pip install plotly -q

# 1)API Scraping

**Création d'un DataFrame des 35 villes avec coordonnées géographiques**

In [2]:
#liste des 35 meilleures villes  selectionnées selon One Week In.com
cities = ["Mont Saint Michel", "St Malo", "Bayeux", "Le Havre","Rouen","Paris","Amiens","Lille",
          "Strasbourg", "Chateau du Haut Koenigsbourg", "Colmar", "Eguisheim", "Besancon", "Dijon",
          "Annecy", "Grenoble", "Lyon", "Gorges", "Bormes les Mimosas", "Cassis", "Marseille", 
          "Aix en Provence", "Avignon", "Uzes", "Nimes", "Aigues Mortes", "Saintes Maries de la mer",
          "Collioure","Carcassonne","Foix","Toulouse","Montauban","Biarritz","Bayonne","La Rochelle"]

In [3]:
# Test sur une ville 
r=requests.get('https://nominatim.openstreetmap.org/?addressdetails=1&q=Annecy&countrycodes=france&format=json')
r.json()[0]

{'place_id': 298516920,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
 'osm_type': 'relation',
 'osm_id': 6791758,
 'boundingbox': ['45.8280024', '45.9766928', '6.0484121', '6.2043932'],
 'lat': '45.8992348',
 'lon': '6.1288847',
 'display_name': 'Annecy, Haute-Savoie, Auvergne-Rhône-Alpes, France métropolitaine, France',
 'class': 'boundary',
 'type': 'administrative',
 'importance': 0.6890769115704565,
 'icon': 'https://nominatim.openstreetmap.org/ui/mapicons/poi_boundary_administrative.p.20.png',
 'address': {'city': 'Annecy',
  'municipality': 'Annecy',
  'county': 'Haute-Savoie',
  'ISO3166-2-lvl6': 'FR-74',
  'state': 'Auvergne-Rhône-Alpes',
  'ISO3166-2-lvl4': 'FR-ARA',
  'region': 'France métropolitaine',
  'country': 'France',
  'country_code': 'fr'}}

In [4]:
print(r.json()[0]['lat'])
print(r.json()[0]['lon'])

45.8992348
6.1288847


In [5]:
# liste de la longitude et la latitude des 35 villes selectionnées 
#grâce à la librairie requests
lat =[]
long =[]
id_city=[]
n=0
for city in cities :
    print('Extraction alt et long city',n+1,' sur 35')
    id_city.append(n)
    n+=1
    r = requests.get(f"https://nominatim.openstreetmap.org/search.php?city={city}&country=france&format=json")
    lat.append(float(r.json()[0]['lat']))
    long.append(float(r.json()[0]['lon']))


Extraction alt et long city 1  sur 35
Extraction alt et long city 2  sur 35
Extraction alt et long city 3  sur 35
Extraction alt et long city 4  sur 35
Extraction alt et long city 5  sur 35
Extraction alt et long city 6  sur 35
Extraction alt et long city 7  sur 35
Extraction alt et long city 8  sur 35
Extraction alt et long city 9  sur 35
Extraction alt et long city 10  sur 35
Extraction alt et long city 11  sur 35
Extraction alt et long city 12  sur 35
Extraction alt et long city 13  sur 35
Extraction alt et long city 14  sur 35
Extraction alt et long city 15  sur 35
Extraction alt et long city 16  sur 35
Extraction alt et long city 17  sur 35
Extraction alt et long city 18  sur 35
Extraction alt et long city 19  sur 35
Extraction alt et long city 20  sur 35
Extraction alt et long city 21  sur 35
Extraction alt et long city 22  sur 35
Extraction alt et long city 23  sur 35
Extraction alt et long city 24  sur 35
Extraction alt et long city 25  sur 35
Extraction alt et long city 26  su

**Création d'un DataFrame contenant pour chaque ville les données météorologiques que nous souhaitons étudier**

In [6]:
## on recupère la météo des 35 villes selectionnées sur 7 prochains jours 
weather =[]
temperature =[]
n=0
for i in range(len(cities)):
    print('Extraction info weather city',n+1,' sur 35')
    r = requests.get(f"https://api.openweathermap.org/data/2.5/onecall?lat={lat[i]}&lon={long[i]}&units=metric&exclude=hourly,current,minutely,alerts&appid=96edad016d12c29b1c6f80594f3bf478")
    l=[]
    t=[]
    for i in range(7):
        l.append(r.json()['daily'][i]['weather'][0]['main'])
        t.append(r.json()['daily'][i]['temp']['min'])
    weather.append(l)
    temperature.append(t)
    n+=1

Extraction info weather city 1  sur 35
Extraction info weather city 2  sur 35
Extraction info weather city 3  sur 35
Extraction info weather city 4  sur 35
Extraction info weather city 5  sur 35
Extraction info weather city 6  sur 35
Extraction info weather city 7  sur 35
Extraction info weather city 8  sur 35
Extraction info weather city 9  sur 35
Extraction info weather city 10  sur 35
Extraction info weather city 11  sur 35
Extraction info weather city 12  sur 35
Extraction info weather city 13  sur 35
Extraction info weather city 14  sur 35
Extraction info weather city 15  sur 35
Extraction info weather city 16  sur 35
Extraction info weather city 17  sur 35
Extraction info weather city 18  sur 35
Extraction info weather city 19  sur 35
Extraction info weather city 20  sur 35
Extraction info weather city 21  sur 35
Extraction info weather city 22  sur 35
Extraction info weather city 23  sur 35
Extraction info weather city 24  sur 35
Extraction info weather city 25  sur 35
Extractio

In [7]:
data ={'city_id': id_city,
       'city_name': cities,
       'city_latitude': lat,
       'city_longitude': long,
       'city_temperature':temperature,
       'city_weather7': weather}
dataset_city = pd.DataFrame(data)
dataset_city['city_temperature'] = dataset_city['city_temperature'].apply(lambda x: sum(x)/len(x))
dataset_city['city_latitude'] = dataset_city['city_latitude'].apply(lambda x: float(x))
dataset_city['city_longitude'] = dataset_city['city_longitude'].apply(lambda x: float(x))
dataset_city.head()

,city_id,city_name,city_latitude,city_longitude,city_temperature,city_weather7
0,0,Mont Saint Michel,48.635954,-1.511460,6.812857,"[Rain, Rain, Rain, Rain, Rain, Clouds, Rain]"
1,1,St Malo,48.649518,-2.026041,8.434286,"[Clouds, Rain, Rain, Rain, Rain, Rain, Rain]"
2,2,Bayeux,49.276462,-0.702474,6.480000,"[Rain, Clouds, Rain, Rain, Rain, Clear, Rain]"
3,3,Le Havre,49.493898,0.107973,8.691429,"[Rain, Rain, Rain, Rain, Rain, Rain, Rain]"
4,4,Rouen,49.440459,1.093966,5.760000,"[Rain, Clouds, Rain, Rain, Rain, Clear, Clouds]"


In [8]:
import plotly.express as px

df = dataset_city
df.describe()
# on ajoute à la colonne température abs(min) parce que ce paramétre 
# n'accepte pas une valeur negative dans la fonction 
# px.scatter_mapbox
size = df['city_temperature']+abs(df['city_temperature'].min())
# creation d'une carte qui affiche la météo des 35 villes  avec scatter_mapbox
fig = px.scatter_mapbox(df,lat='city_latitude', lon='city_longitude',color='city_temperature',size=size,
                 size_max=15, zoom=3, mapbox_style="carto-positron")

fig.show()

In [9]:
## Enregistrement du DataFrame en format csv
dataset_city.to_csv('cities_weather.csv', index=False)

# 2) URL scrapping

scraping des informations suivante:
>* hotel name
>* Url to his booking.com page
>* coordinates: latitude and longitude
>* Score given by the website users
>* Text description of the hotel

In [10]:
#on récupère 20 hotels ainsi que leurs informations en fonction de nos 35 villes selectionnées précédemment 
#grâce a beautifulsoup

#soup = BeautifulSoup("<html>data</html>", "html.parser")
##Utilisation de try, except car certains hotels peuvent ne pas avoir de note car ajoutés récement sur le site.
from bs4 import BeautifulSoup
import requests

def clean_description(description):
    '''
    fonction pour extraire la bonne information concernant description des hotels
    '''
    desc = []
    for x in description:
        xx = x.get('class')
        if xx[0] == 'd8eab2cf7f' and len(xx) == 1 :
            desc.append(x.text)
    return desc

headers = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9'}

hotel_data = []
n=0
for city in cities:
    #print(city)
    url = f"https://www.booking.com/searchresults.fr.html?ss={city}+France"
    r=requests.get(url,headers=headers)
    soup = BeautifulSoup(r.text, 'html.parser')
    list_hotel = soup.find_all("div", class_ = "fcab3ed991 a23c043802")
    description = soup.find_all('div', {'class' : "d8eab2cf7f"})
    desc = clean_description(description)
    lien = soup.find_all("a", class_ = "e13098a59f")   
    score = soup.find_all("div", class_ = "b5cd09854e d10a6220b4") 
    for i in range(10):
        try:
            re = requests.get(f"https://nominatim.openstreetmap.org/search.php?street={list_hotel[i].get_text()}&country=france&format=json")
            latt=float(re.json()[0]['lat'])
            longg= float(re.json()[0]['lon'])
        except:    
            latt  = lat[n]   
            longg = long[n]
            
        raw=[n , list_hotel[i].get_text(), latt , longg , desc[i] , lien[i].get('href') , 
             float(score[i].get_text().replace(',','.')) ]        
        hotel_data.append(raw)
            
    print('Extraction info city ',n+1,' sur 35 city')    
    n+=1;  

header =['city_id','hotel_name','hotel_lat', 'hotel_long','hotel_description','hotel_link','hotel_score']
dataset_hotel = pd.DataFrame(hotel_data,  columns = header)

Extraction info city  1  sur 35 city
Extraction info city  2  sur 35 city
Extraction info city  3  sur 35 city
Extraction info city  4  sur 35 city
Extraction info city  5  sur 35 city
Extraction info city  6  sur 35 city
Extraction info city  7  sur 35 city
Extraction info city  8  sur 35 city
Extraction info city  9  sur 35 city
Extraction info city  10  sur 35 city
Extraction info city  11  sur 35 city
Extraction info city  12  sur 35 city
Extraction info city  13  sur 35 city
Extraction info city  14  sur 35 city
Extraction info city  15  sur 35 city
Extraction info city  16  sur 35 city
Extraction info city  17  sur 35 city
Extraction info city  18  sur 35 city
Extraction info city  19  sur 35 city
Extraction info city  20  sur 35 city
Extraction info city  21  sur 35 city
Extraction info city  22  sur 35 city
Extraction info city  23  sur 35 city
Extraction info city  24  sur 35 city
Extraction info city  25  sur 35 city
Extraction info city  26  sur 35 city
Extraction info city 

In [11]:
# DataFrame contenant les 20 meilleurs hotels de chaque ville selon les recommandations de booking
dataset_hotel.head()

,city_id,hotel_name,hotel_lat,hotel_long,hotel_description,hotel_link,hotel_score
0,0,Le Relais Saint Michel,48.589850,-0.659579,Le Relais Saint Michel vous accueille face à l...,https://www.booking.com/hotel/fr/le-relais-sai...,7.8
1,0,Les Terrasses Poulard,48.636195,-1.509808,Occupant 2 bâtiments différents au cœur du Mon...,https://www.booking.com/hotel/fr/les-terrasses...,7.3
2,0,La Mère Poulard,48.400261,-1.314558,"Occupant un bâtiment historique, l'hôtel La Mè...",https://www.booking.com/hotel/fr/la-mere-poula...,7.0
3,0,Hotel De La Digue,48.616838,-1.511111,L'hôtel De La Digue est un établissement tradi...,https://www.booking.com/hotel/fr/de-la-digue.f...,7.1
4,0,Le Relais Du Roy,48.616309,-1.511077,Le Relais Du Roy est un hôtel 3 étoiles situé ...,https://www.booking.com/hotel/fr/le-relais-du-...,7.9


In [16]:
dataset_hotel.to_csv('cities_hotel.csv', index=False)

In [17]:
# construction du dataset global 
# on fusionne les deux datafram (dataset_hotel et dataset_city)
dataset_kayak = dataset_city.join(dataset_hotel.set_index('city_id'), on ='city_id')
dataset_kayak.head()

,city_id,city_name,city_latitude,city_longitude,city_temperature,city_weather7,hotel_name,hotel_lat,hotel_long,hotel_description,hotel_link,hotel_score
0,0,Mont Saint Michel,48.635954,-1.51146,6.812857,"[Rain, Rain, Rain, Rain, Rain, Clouds, Rain]",Le Relais Saint Michel,48.589850,-0.659579,Le Relais Saint Michel vous accueille face à l...,https://www.booking.com/hotel/fr/le-relais-sai...,7.8
0,0,Mont Saint Michel,48.635954,-1.51146,6.812857,"[Rain, Rain, Rain, Rain, Rain, Clouds, Rain]",Les Terrasses Poulard,48.636195,-1.509808,Occupant 2 bâtiments différents au cœur du Mon...,https://www.booking.com/hotel/fr/les-terrasses...,7.3
0,0,Mont Saint Michel,48.635954,-1.51146,6.812857,"[Rain, Rain, Rain, Rain, Rain, Clouds, Rain]",La Mère Poulard,48.400261,-1.314558,"Occupant un bâtiment historique, l'hôtel La Mè...",https://www.booking.com/hotel/fr/la-mere-poula...,7.0
0,0,Mont Saint Michel,48.635954,-1.51146,6.812857,"[Rain, Rain, Rain, Rain, Rain, Clouds, Rain]",Hotel De La Digue,48.616838,-1.511111,L'hôtel De La Digue est un établissement tradi...,https://www.booking.com/hotel/fr/de-la-digue.f...,7.1
0,0,Mont Saint Michel,48.635954,-1.51146,6.812857,"[Rain, Rain, Rain, Rain, Rain, Clouds, Rain]",Le Relais Du Roy,48.616309,-1.511077,Le Relais Du Roy est un hôtel 3 étoiles situé ...,https://www.booking.com/hotel/fr/le-relais-du-...,7.9


In [18]:
dataset_kayak.to_csv('cities_hotel_wather.csv', index= False)

In [31]:
## Top 5 cities 
top_5_cities= dataset_city.sort_values(by='city_temperature', ascending=False).iloc[:5,:]
top_5_cities

,city_id,city_name,city_latitude,city_longitude,city_temperature,city_weather7
20,20,Marseille,43.296174,5.369953,11.137143,"[Clouds, Clouds, Clouds, Clouds, Clouds, Cloud..."
19,19,Cassis,43.214036,5.539632,11.042857,"[Clouds, Clouds, Clouds, Clouds, Rain, Clouds,..."
26,26,Saintes Maries de la mer,43.452277,4.428717,9.444286,"[Clouds, Clouds, Clouds, Rain, Clouds, Clear, ..."
18,18,Bormes les Mimosas,43.150697,6.341928,9.381429,"[Clouds, Clouds, Clouds, Rain, Rain, Clouds, R..."
27,27,Collioure,42.525050,3.083155,9.291429,"[Rain, Clouds, Clouds, Rain, Clear, Clear, Rain]"


In [20]:
# creation d'une carte qui affiche le top 5 cities avec scatter_mapbox
df = top_5_cities
size = df['city_temperature']

fig = px.scatter_mapbox(df,lat='city_latitude', lon='city_longitude',color='city_temperature',size=size,
                 size_max=15, zoom=3, mapbox_style="carto-positron")

fig.show()

In [21]:
top_5_cities.to_csv('top_5_cities.csv', index= False)

In [22]:
## top 20 hotels
### le critère choisi est celui du score de chaque hotel 
top_20_hotel = dataset_hotel.sort_values(by='hotel_score', ascending=False).iloc[:20,:]
top_20_hotel.head()


,city_id,hotel_name,hotel_lat,hotel_long,hotel_description,hotel_link,hotel_score
161,16,Design and luxurious flat at the heart of Pres...,45.757814,4.832011,Hébergement géré par un particulier,https://www.booking.com/hotel/fr/design-and-lu...,10.0
188,18,Villa Les Alyzés,43.150697,6.341928,Hébergement géré par un particulier,https://www.booking.com/hotel/fr/villa-les-aly...,10.0
130,13,La Maison Des Ducs - Avec Terrasse - Coeur De ...,47.321581,5.041470,Hébergement géré par un particulier,https://www.booking.com/hotel/fr/the-dukes-hou...,10.0
201,20,Charmant Appart / Coeur de Marseille / 5 pers/...,43.296174,5.369953,"In the Le Panier district of Marseille, close ...",https://www.booking.com/hotel/fr/charmant-appa...,10.0
205,20,Appartement Businessman,43.296174,5.369953,"Ideally set in the centre of Marseille, Charma...",https://www.booking.com/hotel/fr/appartement-b...,10.0


In [23]:
##  création d'une carte qui affiche les 20 meilleurs hotels des 35 villes
df = top_20_hotel
size = df['hotel_score']

fig = px.scatter_mapbox(df,lat='hotel_lat', lon='hotel_long',color='hotel_score',size=size,
                 size_max=15, zoom=3, mapbox_style="carto-positron")

fig.show()

In [24]:
top_20_hotel.to_csv('top_20_hotels.csv', index= False)    

# 3) AWS S3

**Push on AWS S3**

In [21]:
!pip install boto3 -q
import boto3
import pandas as pd


In [24]:
## Open session 
session = boto3.Session(aws_access_key_id= '************',
                        aws_secret_access_key= '****************')
# Run session
s3 = session.resource("s3")

maintenant nous allons charger nos csv dans notre bucket(kayak-booking) S3

In [5]:
s3.meta.client.upload_file(Filename = "top_20_hotels.csv", Bucket= "kayak-booking", Key = "kayak-booking")

In [6]:
s3.meta.client.upload_file(Filename = "top_20_hotels.csv", Bucket= "kayak-booking", Key = "kayak-booking.csv")

In [7]:
s3.meta.client.upload_file(Filename = 'cities_hotel_wather.csv', Bucket= "kayak-booking", Key = 'cities_hotel_wather.csv')

**Pull from AWS S3**

In [22]:
from io import BytesIO
import logging

In [12]:
print(obj)

s3.Object(bucket_name='kayak-booking', key='cities_hotel_weather.csv')


In [26]:
obj = s3.Object("kayak-booking", "cities_hotel_wather.csv")# Upload file

In [15]:
file_content = BytesIO(obj.get()['Body'].read())# Reading the File as String With Encoding

In [16]:
dataframe = pd.read_csv(file_content)

In [18]:
dataframe.columns

Index(['city_id', 'city_name', 'city_latitude', 'city_longitude',
       'city_temperature', 'city_weather7', 'hotel_name', 'hotel_lat',
       'hotel_long', 'hotel_description', 'hotel_link', 'hotel_score'],
      dtype='object')

In [19]:
obj = s3.Object("kayak-booking", "kayak-booking.csv")

In [20]:
file_content2 = BytesIO(obj.get()['Body'].read())
dataframe2 = pd.read_csv(file_content2)
dataframe2.columns

Index(['city_id', 'hotel_name', 'hotel_lat', 'hotel_long', 'hotel_description',
       'hotel_link', 'hotel_score'],
      dtype='object')